In [ ]:
import io, pstats, os, json, PIL, cProfile
import matplotlib.pyplot as plt 
import numpy as np 

from src.corrector import DistortionAdjustment_Multiscale, DistortionAdjustment
from src.classGrid import Grid

## Define settings

In [11]:
"""
A large array of SEM images, distortion corrected then stiched
""" 

parameters = {
    # Directory containing input images
    "dire"                 :  'C:\\Users\\mbcx9rt5\\Desktop\\Step_1_small\\', 
    # Offset (padding) in x and y directions (in pixels)
    "ox"                   :  20, 
    "oy"                   :  20,    
    # Interpolation methods for each scale level (coarse to fine): 'bilinear', 'cubic-spline'
    "interpolation_scales" :  ['cubic-spline'],   
    # Subsampling factors for each scale level (reduces resolution for faster computation)
    "subsampling_scales"   :  [1], 
    # Gaussian blur standard deviation for each scale level (noise reduction)
    "sigma_gauss_scales"   :  [0.8],  
    # Number of optimization iterations for each scale level
    "Niter_scales"         :  [10], 
    # Distortion modes: 't' (translation), 'd' (distortion), 't+d' (both)
    "modes"                :  't+d', 
    # Convergence tolerance for optimization
    "tol"                  :  1.e-5,
    # Maximum frequency components (Fourier modes) in x and y for distortion basis
    "mx"                   :  [3, 4, 5, 7, 8], 
    "my"                   :  [3, 4, 5, 6, 9],
    # Initial distortion field (None = start from zero/identity)
    "d0"                   :  None, 
}

results_file = 'identif.json'



## Make grid

In [3]:
# Set up parameters for grid creation
parameters['interpolation'] = parameters['interpolation_scales'][0]
parameters['subsampling'] = parameters['subsampling_scales'][0]
parameters['sigma_gaussian'] = parameters['sigma_gauss_scales'][0]
parameters['Niter'] = parameters['Niter_scales'][0]

# Create grid (auto-detects Field_YY_XX.tif tiles)
grid = Grid.CreateGrid(parameters)

grid.SetTranslations()

Auto-detected grid configuration:
  Grid size: 6 x 4 tiles
  Image size: 2048 x 2048 pixels
  Total tiles found: 24


## Identify distortion field

In [ ]:
""" Running the identification procedure""" 
cam, images, grid, res_tot = DistortionAdjustment_Multiscale(parameters, cam0=None, images0=None, grid0=grid, epsilon=10)
""" Measured distortion"""
cam.ShowValues() 

#%% 
""" Saving the input parameters and results """
results = {
    "projector":  {'mx': cam.mx,
                   'my': cam.my, 
                   'p' : list(cam.p) }  
}
with open(parameters['dire']+results_file, "w") as json_file:
    json.dump(results, json_file)

*********** SCALE 1 ***********
--GN
Iter #  1 | dp/p=1.24e-04 | mean_std=7.30, max_std=14.32
Iter #  2 | dp/p=8.87e-05 | mean_std=6.53, max_std=12.71
Iter #  3 | dp/p=6.07e-05 | mean_std=6.15, max_std=10.46
Iter #  4 | dp/p=3.61e-05 | mean_std=5.95, max_std=8.31
Iter #  5 | dp/p=2.20e-05 | mean_std=5.87, max_std=7.69
Iter #  6 | dp/p=1.54e-05 | mean_std=5.83, max_std=7.65
Iter #  7 | dp/p=1.20e-05 | mean_std=5.81, max_std=7.61
Iter #  8 | dp/p=9.88e-06 | mean_std=5.79, max_std=7.57
X:  x*y  x**2  y**2  x*y**2  x**3  
 [ 1.04058158 -1.80200602 -3.61771315 -0.87632889  0.02694321]
Y:  x*y  x**2  y**2  x**2*y  y**3  
 [-0.94936199  1.2463654   1.93384892  0.09259678  3.99737975]


In [ ]:
prof = cProfile.Profile()
prof.enable()
DistortionAdjustement_Multiscale(parameters, cam0=None, images0=None, grid0=grid, epsilon=10)
prof.disable()

s = io.StringIO()
pstats.Stats(prof, stream=s).sort_stats('cumtime').print_stats(25)
print(s.getvalue())

*********** SCALE 1 ***********
--GN
Iter #  1 | dp/p=1.60e-04 | mean_std=6.39, max_std=8.30
Iter #  2 | dp/p=6.06e-05 | mean_std=5.84, max_std=7.44
Iter #  3 | dp/p=2.73e-05 | mean_std=5.78, max_std=7.47
Iter #  4 | dp/p=1.33e-05 | mean_std=5.77, max_std=7.47
Iter #  5 | dp/p=7.37e-06 | mean_std=5.76, max_std=7.47
         331400 function calls (310357 primitive calls) in 126.699 seconds

   Ordered by: cumulative time
   List reduced from 446 to 25 due to restriction <25>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  385/331    0.252    0.001  605.587    1.830 C:\Users\mbcx9rt5\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3385(mean)
  380/202    0.779    0.002  467.643    2.315 c:\ProgramData\anaconda3\Lib\site-packages\scipy\sparse\_compressed.py:495(_matmul_multivector)
   380/43    0.478    0.001  463.304   10.775 c:\ProgramData\anaconda3\Lib\site-packages\scipy\sparse\_base.py:458(dot)
   380/43    0.433    0.001  377.061  

## Visualise distortion field

In [ ]:
""" Visualization of the measured distortion function """
cam.Plot(size=[parameters['sx'], parameters['sy']]) 
cam.PlotGrid(size=(1024,1024), nelemx=10, alpha=5)
cam.PlotInverseGrid(size=(1024,1024), nelemx=10, alpha=5)
plt.show() 

## Save corrected images

In [ ]:
#%% 
""" Correcting all the images and saving them in a subdirectory  """
save_dire = parameters['dire']+'corrected_'
sigma_gaussian = 0 # Parameter for Gaussian bluring of the images 

x = np.arange(parameters['sx'])
y = np.arange(parameters['sy'])
X,Y = np.meshgrid(x,y,indexing='ij')
xtot = X.ravel() 
ytot = Y.ravel() 

Pxtot,Pytot = cam.P(xtot, ytot)


for i,fname in enumerate(os.listdir(parameters['dire'])):
    print(i,fname)
    if fname[-3:] == 'tif': 
        im_new = corrector.Image(parameters['dire']+fname)
        im_new.Load() 
        if im_new.pix.shape == (parameters['sx'],parameters['sy']): 
            im_new.BuildInterp()
            im_new.GaussianFilter(sigma = sigma_gaussian)
            imc = im_new.Interp(Pxtot, Pytot)
            imc = imc.reshape((parameters['sx'],parameters['sy'])) 
            PILimg = PIL.Image.fromarray(np.round(imc).astype("uint8"))
            PILimg.save(save_dire+fname)  
        else: 
            print("Skip file")
    else: 
        print("Skip file")

## Stitching the mosaic

In [ ]:
ims_unc = grid.StitchImages(cam=None, fusion_mode='linear blending', interpolation_order=1) # Stitching the mosaic without correction 
PILimg = PIL.Image.fromarray(np.round(ims_unc).astype("uint8"))
PILimg.save(parameters['dire']+'mosaic'+'_uncorrected'+parameters['extension'])   

Fusing images (linear blending):
[████████████████████████████████████████]100%



In [6]:
print(cam.p)

[ 1.04058158 -1.80200602 -3.61771315 -0.87632889  0.02694321 -0.94936199
  1.2463654   1.93384892  0.09259678  3.99737975]


In [7]:
ims_c = grid.StitchImages(cam= cam, fusion_mode='linear blending') 
PILimg = PIL.Image.fromarray(np.round(ims_c).astype("uint8"))
PILimg.save(parameters['dire']+'mosaic'+'_corrected'+parameters['extension']) 

Fusing images (linear blending):
[████████████████████████████████████████]100%



In [8]:
grid.ExportTile(parameters['dire']+'TileConfiguration.txt')  # Exporting Tile 

In [ ]:
prof = cProfile.Profile()
prof.enable()
grid.StitchImages(fusion_mode='linear blending', interpolation_order=3)
prof.disable()

s = io.StringIO()
pstats.Stats(prof, stream=s).sort_stats('cumtime').print_stats(25)
print(s.getvalue())

Fusing images (linear blending):
[████████████████████████████████████████]100%

         47004 function calls (45633 primitive calls) in 92.114 seconds

   Ordered by: cumulative time
   List reduced from 300 to 25 due to restriction <25>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       76   20.924    0.275   74.782    0.984 d:\Rhys\Github\Distortion-correction\src\classGrid.py:318(_process_blending_region)
      247    0.599    0.002   51.646    0.209 {method 'run' of '_contextvars.Context' objects}
       66    0.002    0.000   28.403    0.430 c:\ProgramData\anaconda3\Lib\site-packages\scipy\ndimage\_interpolation.py:371(map_coordinates)
       66   27.217    0.412   27.217    0.412 {built-in method scipy.ndimage._nd_image.geometric_transform}
       66   18.870    0.286   26.585    0.403 d:\Rhys\Github\Distortion-correction\src\classProjector.py:28(P)
      208    0.848    0.004   14.694    0.071 c:\ProgramData\anaconda3\Lib\asyncio\windows_events.py:

In [37]:
from classImage import Image

txt = np.loadtxt('C:\\Users\\mbcx9rt5\\Desktop\\Step_1\\Zone_BSE_Step_1_TileReg.txt', skiprows=3, dtype=str)

im = [s[0].split(';')[0] for s in txt]

im= [Image(f) for f in im]

In [29]:
from classGrid import Grid


grid = Grid(nImages=(19,13), overlap=(0.2, 0.2), shape=(19, 13), images=im, regions=247)
grid.ReadTile(file='C:\\Users\\mbcx9rt5\\Desktop\\Step_1\\Zone_BSE_Step_1_TileReg.txt')

KeyboardInterrupt: 